In [1]:
import os
_ROOT_PATH = os.path.join('..', '..', '..')
_FINAL_DFS_DIR = os.path.join('..', '..', 'final_dfs')
_CSV_DIR = os.path.join('..', '..', 'csv_files')

import sys
sys.path.append('..')
sys.path.append(os.path.join('..', '..', '..', 'tools'))
import plotconfig

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df_file = os.path.join(_FINAL_DFS_DIR, 'optim_rmse_y_true_vs_y_reconstructed.parquet')
result_df = pd.read_parquet(df_file, engine='pyarrow')

In [4]:
result_df.iloc[0]['true_labels'].shape

(9234,)

In [5]:
9234*170

1569780

In [6]:
# Assuming your DataFrame is named 'df'
grouped_df = result_df.groupby(['method_name'])

# Calculate mean and std for each subtable
compiled_result = grouped_df.agg({
    'rmse': ['mean', 'std']
})

# Flatten the column names
compiled_result.columns = ['_'.join(col).strip() for col in compiled_result.columns.values]

# Reset the index to display method_name and training_size as columns
compiled_result = compiled_result.reset_index()
compiled_result

,method_name,rmse_mean,rmse_std
0,DummyScoring_Mean,18.231405,2.986684
1,LinearRegression,0.176395,0.079547
2,Shuffle_LinearRegression,15.200366,3.206717


In [7]:
# Dictionary to store rmse values for each method
rmse_values = {}
true_labels = {}
pred_labels = {}

# Loop through each unique method name
for method in result_df['method_name'].unique():
    # Create a mask for the current method
    mask = (result_df['method_name'] == method)
    
    # Filter the DataFrame and get the 'rmse' values
    df = result_df[mask]
    rmse_values[method] = df['rmse'].values
    true_labels[method] = df['true_labels'].values
    pred_labels[method] = df['pred_labels'].values

In [8]:
true_labels.keys()

dict_keys(['LinearRegression', 'Shuffle_LinearRegression', 'DummyScoring_Mean'])

In [9]:
flattened_true_labels = {method: [label for sublist in labels for label in sublist] for method, labels in true_labels.items()}

In [10]:
len(flattened_true_labels['LinearRegression'])

1569780

In [11]:
directory = '../../../user_experiments/results'

dfs = []
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path)
        dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

df['match'] = (df['target'] != df['chosen']).astype(int)
mask = ((df['showing_time'] == 500))

df_masked = df[mask]

grouped_df = df_masked.groupby(['rounded_d', 'showing_time'])
compiled_result = grouped_df.agg({
    'match': ['mean', 'std', 'sem', 'count']
})
compiled_result.columns = ['_'.join(col).strip() for col in compiled_result.columns.values]
compiled_result = compiled_result.reset_index()
compiled_result

,rounded_d,showing_time,match_mean,match_std,match_sem,match_count
0,0.0,500,0.493243,0.501652,0.041236,148
1,1.6,500,0.503226,0.501610,0.040290,155
2,3.6,500,0.376623,0.486120,0.039173,154
3,6.8,500,0.289017,0.454622,0.034564,173
4,10.9,500,0.129496,0.336963,0.028581,139
5,16.3,500,0.036649,0.188393,0.013632,191
6,21.7,500,0.031646,0.175611,0.013971,158
7,29.4,500,0.006452,0.080322,0.006452,155
8,36.7,500,0.000000,0.000000,0.000000,165
9,46.2,500,0.006579,0.081111,0.006579,152


In [12]:

# Dictionary to store rmse values for each method
match_values = {}

# Loop through each unique method name
for rd in df_masked['rounded_d'].unique():
    # Create a mask for the current method
    mask = (df_masked['rounded_d'] == rd)
    
    # Filter the DataFrame and get the 'rmse' values
    df = df_masked[mask]
    match_values[rd] = df['match'].values

match_values.keys()

dict_keys([np.float64(1.6), np.float64(0.0), np.float64(46.2), np.float64(29.4), np.float64(3.6), np.float64(16.3), np.float64(10.9), np.float64(36.7), np.float64(21.7), np.float64(6.8)])

In [13]:
import numpy as np
from scipy.stats import fisher_exact
from itertools import combinations

def pairwise_fisher_exact(match_values):
    keys = list(match_values.keys())
    results = {}

    for (key1, key2) in combinations(keys, 2):
        x1 = match_values[key1].sum()  # successes in group 1
        n1 = len(match_values[key1])   # trials in group 1
        x2 = match_values[key2].sum()  # successes in group 2
        n2 = len(match_values[key2])   # trials in group 2

        contingency_table = [[x1, n1 - x1], [x2, n2 - x2]]
        
        # Perform Fisher's Exact Test
        odds_ratio, p_value = fisher_exact(contingency_table, alternative='two-sided')
        
        results[(key1, key2)] = {
            'odds_ratio': odds_ratio,
            'p_value': p_value
        }

    return results

results = pairwise_fisher_exact(match_values)

# Print results
# for (key1, key2), stats in results.items():
#     print(f"Comparison between {key1} and {key2}:")
#     print(f"  Odds Ratio: {stats['odds_ratio']}")
#     print(f"  P-value: {stats['p_value']}")
#     print()

In [14]:
results[(3.6, 21.7)]

{'odds_ratio': np.float64(18.4875),
 'p_value': np.float64(1.7242598757797728e-15)}